In [1]:
import pandas as pd
import os
import detect_simpsons_paradox as dsp
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [2]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the labeledDataFrame with that DataFrame to create the object

In [3]:
labeled_df_setup = dsp.labeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [4]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [5]:
roles = {'state':['trend','groupby'], 'year':'trend', 'driver_gender':['trend','groupby'], 'driver_race':['trend','groupby'],
       'decriminalization':['groupby'], 'medical':['groupby'],
         'recreational':['groupby'], 'no_reforms':['groupby'],
       'search_conducted_false':'ignore', 'search_conducted_true':'ignore',
       'search_conducted_rate':'trend', 'contraband_found_false':'ignore',
       'contraband_found_true':'ignore', 'contraband_found_rate':'trend', 'hit_false':'ignore',
       'hit_true':'ignore', 'hit_rate':'trend', 'num_stops':'trend'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
var_types = {'driver_gender':'categorical'}
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops','contraband_found_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [6]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.set_weighting_vars(weighting)
labeled_df_setup.set_var_types(var_types)
labeled_df_setup.meta_df

/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['role'][k] = v
/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['var_type'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,trend,False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,ignore,True,NaN


Now, we've set this up, we can also save these configurations to load them in directly in the future

In [7]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

We can see what this does, using a bash magic

In [8]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [9]:
labeled_df = dsp.labeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [10]:
labeled_df.add_all_dpgmm(qual_thresh =-1.5)
labeled_df.df.head()

/home/smb/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


In [11]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN
search_conducted_true,float64,continuous,[ignore],True,NaN


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [12]:
corrobj = dsp.all_pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

['year',
 'search_conducted_rate',
 'contraband_found_rate',
 'hit_rate',
 'num_stops']

In [13]:
rankobj = dsp.mean_rank_trend()
linreg_obj = dsp.all_linear_trend()

# Computing Trends on a labeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [14]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()

,feat1,feat2,subgroup_trend,subgroup_trend_quality,group_feat,subgroup,trend_type,agg_trend,agg_trend_quality
0,year,search_conducted_rate,-0.247018,-0.247018,state,AZ,pearson_corr,-0.03903,-0.03903
1,year,search_conducted_rate,-0.414566,-0.414566,state,CO,pearson_corr,-0.03903,-0.03903
2,year,search_conducted_rate,0.118238,0.118238,state,CT,pearson_corr,-0.03903,-0.03903
3,year,search_conducted_rate,-0.199765,-0.199765,state,IL,pearson_corr,-0.03903,-0.03903
4,year,search_conducted_rate,-0.603026,-0.603026,state,MA,pearson_corr,-0.03903,-0.03903


Now we can use a list of objects and apply multiple trends

In [15]:
labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality
56,contraband_found_rate,driver_gender,state,RI,"[F, M]",0.511200,rank_trend,"[F, M]",0.179700
280,year,search_conducted_rate,recreational,0,-0.000213106,0.022852,lin_reg,-0.000353381,0.039030
442,search_conducted_rate,num_stops,driver_race,Other,9435.64,0.061633,lin_reg,-257201,0.065594
172,num_stops,driver_gender,no_reforms,1,"[F, M]",0.251300,rank_trend,"[F, M]",0.116400
353,year,num_stops,driver_gender,F,-793.986,0.036268,lin_reg,-1560.86,0.043965
78,contraband_found_rate,driver_race,state,IL,"[Asian, Other, White, Hispanic, Black]",0.398900,rank_trend,"[Asian, White, Other, Hispanic, Black]",0.071600
201,search_conducted_rate,state,driver_race,Other,"[CO, NC, MA, CT, MD, SC, RI, TX, IL, WI, AZ, V...",0.621800,rank_trend,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, WA, RI, I...",0.388100
272,year,search_conducted_rate,driver_race,Black,-0.000555503,0.039174,lin_reg,-0.000353381,0.039030
363,year,num_stops,medical,1,925.637,0.031944,lin_reg,-1560.86,0.043965
275,year,search_conducted_rate,driver_race,White,-0.000402739,0.115382,lin_reg,-0.000353381,0.039030


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

We can see what types of trends were computed from `result_df`

In [16]:
pd.unique(labeled_df.result_df['trend_type'])

array(['rank_trend', 'lin_reg'], dtype=object)

The object also stores the trend objects that have been applied, they can be used for mapping to get the distance functions that are appropriate for each trend

In [17]:
labeled_df.trend_list

In [18]:
# labeled_df.result_df['distance'] = labeled_df.result_df.apply(dist_helper,axis=1)
labeled_df.add_distance()

labeled_df.result_df.sample(10)

/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3559: RuntimeWarning: invalid value encountered in double_scalars
  size * (size - 1) * (size - 2))


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance
425,search_conducted_rate,num_stops,state,CO,-1.80108e+06,0.101575,lin_reg,-257201,0.065594,0.000191
200,search_conducted_rate,state,driver_race,Hispanic,"[CO, NC, MD, TX, WI, CT, MA, VT, WA, RI, AZ, S...",0.500600,rank_trend,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, WA, RI, I...",0.388100,0.360000
58,contraband_found_rate,driver_gender,state,TX,"[F, M]",0.428700,rank_trend,"[F, M]",0.179700,0.000000
441,search_conducted_rate,num_stops,driver_race,Hispanic,-213893,0.062212,lin_reg,-257201,0.065594,0.000045
203,search_conducted_rate,state,decriminalization,0,"[VT, MD, WI, SC, WA, MA, IL, RI, AZ]",0.070100,rank_trend,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, WA, RI, I...",0.388100,0.290000
54,contraband_found_rate,driver_gender,state,MD,"[F, M]",0.262600,rank_trend,"[F, M]",0.179700,0.000000
364,year,num_stops,recreational,0,-1465.04,0.040337,lin_reg,-1560.86,0.043965,0.002401
100,hit_rate,driver_gender,state,CT,"[F, M]",0.180600,rank_trend,"[F, M]",0.043300,0.000000
156,num_stops,driver_gender,state,TX,"[F, M]",0.200000,rank_trend,"[F, M]",0.116400,0.000000
165,num_stops,driver_gender,decriminalization,0,"[F, M]",0.160300,rank_trend,"[F, M]",0.116400,0.000000


Each trend object has a trend_precompute dictionary as a property that stores the intermediate values (tables of the weighted rates for ranks and correlation matrices for pearson correlation, TODO: what do we need for linreg). These can be used in vizualization.

In [19]:
labeled_df.trend_list[0].trend_precompute

{'rank_trend_agg_trend_contraband_found_rate_driver_gender': driver_gender
 F    0.003184
 M    0.006874
 dtype: float64,
 'rank_trend_agg_trend_contraband_found_rate_driver_race': driver_race
 Asian       0.002592
 White       0.004982
 Other       0.005700
 Hispanic    0.006455
 Black       0.008367
 dtype: float64,
 'rank_trend_agg_trend_contraband_found_rate_state': state
 NC    0.001062
 CO    0.002303
 MD    0.003503
 WA    0.004353
 CT    0.005716
 SC    0.006382
 TX    0.006541
 WI    0.007934
 IL    0.008215
 MA    0.008255
 VT    0.009567
 AZ    0.009980
 RI    0.013719
 dtype: float64,
 'rank_trend_agg_trend_hit_rate_driver_gender': driver_gender
 F    0.243620
 M    0.256954
 dtype: float64,
 'rank_trend_agg_trend_hit_rate_driver_race': driver_race
 Asian       0.165508
 Hispanic    0.193965
 Other       0.195967
 Black       0.269228
 White       0.279382
 dtype: float64,
 'rank_trend_agg_trend_hit_rate_state': state
 WA    0.135670
 NC    0.146822
 IL    0.205232
 AZ    0

# Filtering

In [20]:
help(labeled_df.get_trend_rows)

Help on method get_trend_rows in module detect_simpsons_paradox.ranking_processing:

get_trend_rows(feat1=None, feat2=None, group_feat=None, subgroup=None) method of detect_simpsons_paradox.labeled_dataframe.labeledDataFrame instance
    return a row of result_df based on the specified values. returned rows
    meet provided criteria for all columns (and operator) and any one of the listed
    values for each column (or operator)
    
    Parameters
    -----------
    feat1 : str, list, or  None
        trend variable name or None to include all
    feat2 : str, list, or  None
        trend variable name or None to include all
    group_feat : str, list, or  None
        groupoby variable name or None to include all
    subgroup : str, list, or  None
        value of groupby_feat or or None to include all



So, we can use that function to filter and look at subsets of the trends based on the features, groupby, or subgroups

In [21]:
labeled_df.get_trend_rows(feat1='year',subgroup=['Black','Hispanic'])

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance
272,year,search_conducted_rate,driver_race,Black,-0.000555503,0.039174,lin_reg,-0.000353381,0.039030,0.011581
273,year,search_conducted_rate,driver_race,Hispanic,-0.0014828,0.177694,lin_reg,-0.000353381,0.039030,0.064711
356,year,num_stops,driver_race,Black,-2262.72,0.169689,lin_reg,-1560.86,0.043965,0.011386
357,year,num_stops,driver_race,Hispanic,422.667,0.014732,lin_reg,-1560.86,0.043965,179.827735


We can also filter based on SP

In [22]:
labeled_df.get_SP_rows(thresh=.2)

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh_0.2
28,search_conducted_rate,driver_race,state,CT,"[Asian, Other, White, Hispanic, Black]",0.443000,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.400000,True
30,search_conducted_rate,driver_race,state,MA,"[Other, Asian, White, Black, Hispanic]",0.304800,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.400000,True
33,search_conducted_rate,driver_race,state,RI,"[Other, Asian, White, Hispanic, Black]",0.477600,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.600000,True
34,search_conducted_rate,driver_race,state,SC,"[Other, White, Black, Hispanic]",0.414300,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.260000,True
36,search_conducted_rate,driver_race,state,VT,"[Asian, White, Hispanic, Black, Other]",0.022600,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.600000,True
37,search_conducted_rate,driver_race,state,WA,"[Asian, White, Hispanic, Black, Other]",0.528600,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.600000,True
38,search_conducted_rate,driver_race,state,WI,"[Asian, White, Hispanic, Other, Black]",0.244700,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.400000,True
39,search_conducted_rate,driver_race,driver_gender,F,"[Asian, White, Black, Hispanic, Other]",0.213900,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.400000,True
42,search_conducted_rate,driver_race,decriminalization,1,"[Other, Asian, White, Black, Hispanic]",0.151500,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.400000,True
43,search_conducted_rate,driver_race,medical,0,"[Other, Asian, White, Black, Hispanic]",0.189200,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.400000,True


# Ranking

In [23]:
labeled_df.rank_occurences_by_view(ascending=False).head(20)

                    feat1                  feat2  mean_view_distance
0   contraband_found_rate          driver_gender            0.000000
1   contraband_found_rate            driver_race            0.272609
2   contraband_found_rate                  state            0.219333
3                hit_rate          driver_gender            0.384615
4                hit_rate            driver_race            0.367826
5                hit_rate                  state            0.172667
6               num_stops          driver_gender            0.000000
7               num_stops            driver_race            0.176522
8               num_stops                  state            0.236667
9   search_conducted_rate          driver_gender            0.000000
10  search_conducted_rate            driver_race            0.341739
11  search_conducted_rate              num_stops           45.000919
12  search_conducted_rate                  state            0.245333
13                   year         

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh_0.2,mean_view_distance
284,year,num_stops,state,AZ,8474.29,0.301916,lin_reg,-1560.86,0.043965,179.956531,True,77.068621
288,year,num_stops,state,MA,2227.27,0.115809,lin_reg,-1560.86,0.043965,179.937568,True,77.068621
290,year,num_stops,state,NC,2120.79,0.117245,lin_reg,-1560.86,0.043965,179.936276,True,77.068621
306,year,num_stops,medical,0,1794.75,0.044001,lin_reg,-1560.86,0.043965,179.931368,True,77.068621
311,year,num_stops,no_reforms,1,1309.45,0.044523,lin_reg,-1560.86,0.043965,179.919537,True,77.068621
307,year,num_stops,medical,1,925.637,0.031944,lin_reg,-1560.86,0.043965,179.901393,True,77.068621
304,year,num_stops,decriminalization,0,637.751,0.025548,lin_reg,-1560.86,0.043965,179.873452,True,77.068621
301,year,num_stops,driver_race,Hispanic,422.667,0.014732,lin_reg,-1560.86,0.043965,179.827735,True,77.068621
294,year,num_stops,state,VT,271.488,0.044411,lin_reg,-1560.86,0.043965,179.752250,True,77.068621
299,year,num_stops,driver_race,Asian,259.628,0.104919,lin_reg,-1560.86,0.043965,179.742609,True,77.068621


In [24]:
labeled_df.add_view_score('SP_thresh_0.2',agg_type='sum',colored=False)

                    feat1                  feat2  sum_view_SP_thresh_0.2
0   contraband_found_rate          driver_gender                     0.0
1   contraband_found_rate            driver_race                    12.0
2   contraband_found_rate                  state                     8.0
3                hit_rate          driver_gender                     5.0
4                hit_rate            driver_race                    13.0
5                hit_rate                  state                     4.0
6               num_stops          driver_gender                     0.0
7               num_stops            driver_race                     5.0
8               num_stops                  state                     8.0
9   search_conducted_rate          driver_gender                     0.0
10  search_conducted_rate            driver_race                    14.0
11  search_conducted_rate              num_stops                     7.0
12  search_conducted_rate                  state   

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh_0.2,mean_view_distance,sum_view_SP_thresh_0.2
0,year,num_stops,state,AZ,8474.29,0.301916,lin_reg,-1560.86,0.043965,179.956531,True,77.068621,12.0
1,year,num_stops,state,MA,2227.27,0.115809,lin_reg,-1560.86,0.043965,179.937568,True,77.068621,12.0
2,year,num_stops,state,NC,2120.79,0.117245,lin_reg,-1560.86,0.043965,179.936276,True,77.068621,12.0
3,year,num_stops,medical,0,1794.75,0.044001,lin_reg,-1560.86,0.043965,179.931368,True,77.068621,12.0
4,year,num_stops,no_reforms,1,1309.45,0.044523,lin_reg,-1560.86,0.043965,179.919537,True,77.068621,12.0
5,year,num_stops,medical,1,925.637,0.031944,lin_reg,-1560.86,0.043965,179.901393,True,77.068621,12.0
6,year,num_stops,decriminalization,0,637.751,0.025548,lin_reg,-1560.86,0.043965,179.873452,True,77.068621,12.0
7,year,num_stops,driver_race,Hispanic,422.667,0.014732,lin_reg,-1560.86,0.043965,179.827735,True,77.068621,12.0
8,year,num_stops,state,VT,271.488,0.044411,lin_reg,-1560.86,0.043965,179.752250,True,77.068621,12.0
9,year,num_stops,driver_race,Asian,259.628,0.104919,lin_reg,-1560.86,0.043965,179.742609,True,77.068621,12.0


In [25]:
labeled_df.rank_occurences_by_view('sum_view_SP_thresh_0.2','SP_thresh_0.2').head()

Index(['feat1', 'feat2', 'group_feat', 'subgroup', 'subgroup_trend',
       'subgroup_trend_quality', 'trend_type', 'agg_trend',
       'agg_trend_quality', 'distance', 'SP_thresh_0.2', 'mean_view_distance',
       'sum_view_SP_thresh_0.2'],
      dtype='object')


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh_0.2,mean_view_distance,sum_view_SP_thresh_0.2
105,search_conducted_rate,driver_race,state,RI,"[Other, Asian, White, Hispanic, Black]",0.4776,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.2772,0.6,True,0.341739,14.0
106,search_conducted_rate,driver_race,state,VT,"[Asian, White, Hispanic, Black, Other]",0.0226,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.2772,0.6,True,0.341739,14.0
107,search_conducted_rate,driver_race,state,WA,"[Asian, White, Hispanic, Black, Other]",0.5286,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.2772,0.6,True,0.341739,14.0
108,search_conducted_rate,driver_race,recreational,1,"[Asian, White, Hispanic, Black, Other]",0.4920,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.2772,0.6,True,0.341739,14.0
109,search_conducted_rate,driver_race,no_reforms,1,"[Other, White, Asian, Black, Hispanic]",0.3005,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.2772,0.6,True,0.341739,14.0
